# Documentation website scraper 

1. Specify the source Sitemap for documentation
2. Specify the target desitnation
3. Retrieve the Lost modified value along with all valid urls to be scraped
4. Scrape the content from the URL only if the the corresponding record doesn't exist or if the last mod value of url is greater than the lastmod value in storage ( indicating that docs have been updated for that URL)
5. Run scraping for multiple pages in parallel
6. Store the html content and also the translated markdown content in the target table
7. Exist when no new urls are left

In [ ]:
import os
import time
import datetime
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import asyncio
import aiohttp
import nest_asyncio
import traceback
import html2text
from concurrent.futures import ThreadPoolExecutor
from functools import partial
import singlestoredb as s2
from singlestoredb.management import get_secret

In [ ]:
# Retrieve credentials from environment / secrets
TARGET_HOST = get_secret('host')
TARGET_USER = get_secret('user')
TARGET_PASSWORD = get_secret('ad_pwd')
TARGET_DATABASE = "knowlagent"
TABLE_NAME = "s2docs"
CONCURRENCY = 10
SPIDER_SITEMAP_URL = "https://docs.singlestore.com/sitemap-0.xml"

In [ ]:
def execute_query(query, params=None, commit=False):
    """
    Execute a query on the SingleStore database. Closes connection after.
    """
    connection = s2.connect(
        host=TARGET_HOST,
        port='3306',
        user=TARGET_USER,
        password=TARGET_PASSWORD,
        database=TARGET_DATABASE
    )
    result = None
    try:
        with connection.cursor() as cur:
            if params:
                cur.execute(query, params)
            else:
                cur.execute(query)
            if commit:
                connection.commit()
            else:
                result = cur.fetchall()
    except Exception as e:
        tb = traceback.format_exc()
        print(f"Error executing query: {e}\n{tb}")
    finally:
        connection.close()
    return result


def create_table():
    """
    Create the documents table if it does not exist.
    The table stores:
      - id: primary key
      - url: scraped URL (as TEXT, to allow very long URLs)
      - content: full HTML content
      - md_content:  parsed content in markdown
      - scraped_at: UTC timestamp
      - lastmod: when the content was last modified on the source site
    """
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
        id INT AUTO_INCREMENT PRIMARY KEY,
        url TEXT,
        content LONGTEXT,
        md_content LONGTEXT NULL,
        scraped_at DATETIME,
        lastmod DATETIME
    );
    """
    execute_query(create_table_query, commit=True)
    print(f"Table '{TABLE_NAME}' is ready.")


def get_sitemap_urls(sitemap_url):
    """
    Retrieve and parse the sitemap XML to extract all URLs listed with their lastmod dates.
    Returns a list of tuples (url, lastmod) where lastmod is a datetime object or None.
    """
    try:
        response = requests.get(sitemap_url, timeout=10)
        response.raise_for_status()
        root = ET.fromstring(response.content)
        url_data = []
        
        # Try without namespace first
        for url_elem in root.findall('.//url'):
            loc = url_elem.find('loc')
            lastmod_elem = url_elem.find('lastmod')
            
            if loc is not None and loc.text:
                url = loc.text.strip()
                lastmod = None
                
                # Parse lastmod if available
                if lastmod_elem is not None and lastmod_elem.text:
                    try:
                        # Try to parse the date format in the sitemap
                        lastmod_str = lastmod_elem.text.strip()
                        # Handle common date formats
                        if 'T' in lastmod_str:
                            # ISO format with timezone
                            lastmod = datetime.datetime.fromisoformat(lastmod_str.replace('Z', '+00:00'))
                        else:
                            # Simple YYYY-MM-DD format
                            lastmod = datetime.datetime.strptime(lastmod_str, '%Y-%m-%d')
                    except ValueError:
                        print(f"Could not parse lastmod date for {url}: {lastmod_elem.text}")
                
                url_data.append((url, lastmod))
                
        # If we didn't find any URLs, try with namespace
        if not url_data:
            namespace = {'ns': root.tag.split('}')[0].strip('{')}
            for url_elem in root.findall('ns:url', namespace):
                loc = url_elem.find('ns:loc', namespace)
                lastmod_elem = url_elem.find('ns:lastmod', namespace)
                
                if loc is not None and loc.text:
                    url = loc.text.strip()
                    lastmod = None
                    
                    # Parse lastmod if available
                    if lastmod_elem is not None and lastmod_elem.text:
                        try:
                            lastmod_str = lastmod_elem.text.strip()
                            if 'T' in lastmod_str:
                                lastmod = datetime.datetime.fromisoformat(lastmod_str.replace('Z', '+00:00'))
                            else:
                                lastmod = datetime.datetime.strptime(lastmod_str, '%Y-%m-%d')
                        except ValueError:
                            print(f"Could not parse lastmod date for {url}: {lastmod_elem.text}")
                    
                    url_data.append((url, lastmod))
                    
        return url_data
    except Exception as e:
        tb = traceback.format_exc()
        print(f"Error fetching or parsing sitemap: {e}\n{tb}")
        return []


def get_scraped_urls_with_lastmod():
    """
    Batch query: get all URLs that have already been scraped with their lastmod date.
    Returns a dictionary mapping URL to lastmod datetime.
    """
    query = f"SELECT url, lastmod FROM {TABLE_NAME};"
    rows = execute_query(query)
    return {row[0]: row[1] for row in rows} if rows else {}


def convert_html_to_markdown(html_content):
    """
    Convert HTML content to markdown format.
    
    Args:
        html_content (str): HTML content to convert
        
    Returns:
        str: Markdown formatted content
    """
    try:
        # Configure html2text
        h = html2text.HTML2Text()
        h.ignore_links = False
        h.ignore_images = False
        h.ignore_tables = False
        h.body_width = 0  # No wrapping
        h.unicode_snob = True  # Use Unicode instead of ASCII
        h.single_line_break = True  # Use single line breaks
        
        # Convert HTML to markdown
        markdown_content = h.handle(html_content)
        return markdown_content
    except Exception as e:
        tb = traceback.format_exc()
        print(f"Error converting HTML to markdown: {e}\n{tb}")
        return None


def insert_page(page_url, content, lastmod=None):
    """
    Insert the scraped page into the database with lastmod information and markdown content.
    """
    now = datetime.datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
    
    # If lastmod is None, use the current time
    if lastmod is None:
        lastmod = now
    elif isinstance(lastmod, datetime.datetime):
        lastmod = lastmod.strftime('%Y-%m-%d %H:%M:%S')
    
    # Convert HTML content to markdown
    md_content = convert_html_to_markdown(content)
    
    # Insert with markdown content
    query = f"INSERT INTO {TABLE_NAME} (url, content, scraped_at, lastmod, md_content) VALUES (%s, %s, %s, %s, %s)"
    try:
        execute_query(query, (page_url, content, now, lastmod, md_content), commit=True)
    except Exception as e:
        tb = traceback.format_exc()
        print(f"Error inserting {page_url}: {e}\n{tb}")


async def async_scrape_page(session, page_url, retries=3):
    """
    Asynchronously scrape page content using aiohttp with a retry mechanism in case of timeouts/errors.
    """
    for attempt in range(1, retries+1):
        try:
            # Increase timeout from 10 to 30 seconds
            async with session.get(page_url, timeout=30) as response:
                response.raise_for_status()
                text = await response.text()
                soup = BeautifulSoup(text, 'html.parser')
                return page_url, soup.prettify()
        except asyncio.TimeoutError as te:
            tb = traceback.format_exc()
            print(f"TimeoutError scraping {page_url} on attempt {attempt}: {te}\n{tb}")
        except Exception as e:
            tb = traceback.format_exc()
            print(f"Error scraping {page_url} on attempt {attempt}: {e}\n{tb}")
        # Wait before retrying
        await asyncio.sleep(2)
    # After retries, return None
    return page_url, None


async def async_run_scraper(concurrency=CONCURRENCY,SPIDER_SITEMAP_URL=SPIDER_SITEMAP_URL):
    """
    Main async function that:
      1. Retrieves the sitemap URLs and batch-checks already scraped URLs.
      2. Scrapes pending pages concurrently using aiohttp, limited by a semaphore for concurrency.
      3. Only scrapes pages that are new or have been updated since last scraped.
      4. Prints progress updates every 1% of pending pages processed.
    """
    url_data = get_sitemap_urls(SPIDER_SITEMAP_URL)
    total = len(url_data)
    print(f"Found {total} URLs in sitemap.")

    # Get existing URLs with their lastmod dates
    scraped_data = get_scraped_urls_with_lastmod()
    
    # Filter URLs that need scraping (new or updated)
    pending_urls = []
    for url, lastmod in url_data:
        if url not in scraped_data:
            # New URL, needs scraping
            pending_urls.append((url, lastmod))
        elif lastmod is not None and scraped_data[url] is not None:
            # URL exists, check if it's been updated
            if lastmod > scraped_data[url]:
                # Content has been updated, needs re-scraping
                pending_urls.append((url, lastmod))
    
    pending_total = len(pending_urls)
    print(f"{pending_total} URLs pending scraping (after checking for updates).")

    last_percent = 0
    sem = asyncio.Semaphore(concurrency)

    async def bounded_scrape(url_info, session):
        url, lastmod = url_info
        async with sem:
            return await async_scrape_page(session, url), lastmod

    async with aiohttp.ClientSession() as session:
        tasks = [bounded_scrape(url_info, session) for url_info in pending_urls]
        for idx, future in enumerate(asyncio.as_completed(tasks), start=1):
            (page_url, content), lastmod = await future
            if content:
                insert_page(page_url, content, lastmod)
            
            current_percent = (idx / pending_total) * 100
            if current_percent - last_percent >= 1 or idx == pending_total:
                print(f"Progress: {current_percent:.1f}% ({idx}/{pending_total})")
                last_percent = current_percent


def run_scraper():
    """
    Run the async scraper.
    """
    try:
        asyncio.run(async_run_scraper())
    except RuntimeError as e:
        # If an event loop is already running, apply nest_asyncio to allow nested loops
        import nest_asyncio
        nest_asyncio.apply()
        asyncio.run(async_run_scraper())


def update_lastmod_from_sitemap():
    """
    Update the lastmod column values in the database to match those in the sitemap.
    Processes in batches to avoid memory issues.
    """
    print("Updating lastmod values from sitemap...")
    SPIDER_SITEMAP_URL = "https://docs.singlestore.com/sitemap-0.xml"
    
    # Get all URLs and their lastmod values from the sitemap
    url_data = get_sitemap_urls(SPIDER_SITEMAP_URL)
    if not url_data:
        print("No URLs found in sitemap or error parsing sitemap.")
        return
    
    # Get a dictionary of URLs in the database
    query = f"SELECT id, url FROM {TABLE_NAME};"
    db_rows = execute_query(query)
    if not db_rows:
        print("No URLs found in database.")
        return
        
    # Create a lookup from URL to id
    db_url_to_id = {row[1]: row[0] for row in db_rows}
    
    # Update lastmod values in batches
    batch_size = 100
    update_count = 0
    total_urls = len(url_data)
    print(f"Found {total_urls} URLs in sitemap.")
    
    # Group updates in batches
    batches = []
    current_batch = []
    
    for url, lastmod in url_data:
        if url in db_url_to_id and lastmod is not None:
            current_batch.append((db_url_to_id[url], lastmod))
            if len(current_batch) >= batch_size:
                batches.append(current_batch)
                current_batch = []
    
    # Add any remaining items
    if current_batch:
        batches.append(current_batch)
    
    # Process each batch
    for batch_idx, batch in enumerate(batches, 1):
        # Generate update query with multiple values
        update_values = []
        for id_val, lastmod in batch:
            lastmod_str = lastmod.strftime('%Y-%m-%d %H:%M:%S')
            update_values.append(f"WHEN {id_val} THEN '{lastmod_str}'")
        
        id_list = ', '.join(str(id_val) for id_val, _ in batch)
        update_query = f"""
        UPDATE {TABLE_NAME} 
        SET lastmod = CASE id 
            {' '.join(update_values)}
        END
        WHERE id IN ({id_list});
        """
        
        try:
            execute_query(update_query, commit=True)
            update_count += len(batch)
            print(f"Processed batch {batch_idx}/{len(batches)} ({update_count}/{total_urls} URLs)")
            # Small delay to avoid memory pressure
            time.sleep(0.5)
        except Exception as e:
            tb = traceback.format_exc()
            print(f"Error updating batch {batch_idx}: {e}\n{tb}")
    
    print(f"Updated lastmod values for {update_count} URLs from sitemap.")

In [ ]:
# prepare target tables if they dont exist
create_table()
# Run scraper (e.g. via cron), using asynchronous scraping.
run_scraper()
print("Scraping complete.")